The idea of this notebook borrwos from ClipCap (https://arxiv.org/abs/2111.09734v1). Using encoder to translate image into vector space, and use this vector concatenate with captions to load into GPT-2sequenceClassification model. The output would be the probability of image and caption match.

In [ ]:
import os
if not os.path.isfile("/content/captions.txt"):
  !unzip /content/drive/MyDrive/'Colab Notebooks'/stat940/stat940_final_project/flickr8k.zip

Streaming output truncated to the last 5000 lines.
  inflating: Images/2844963839_ff09cdb81f.jpg  
  inflating: Images/2845246160_d0d1bbd6f0.jpg  
  inflating: Images/2845691057_d4ab89d889.jpg  
  inflating: Images/2845845721_d0bc113ff7.jpg  
  inflating: Images/2846037553_1a1de50709.jpg  
  inflating: Images/2846785268_904c5fcf9f.jpg  
  inflating: Images/2846843520_b0e6211478.jpg  
  inflating: Images/2847514745_9a35493023.jpg  
  inflating: Images/2847615962_c330bded6e.jpg  
  inflating: Images/2847859796_4d9cb0d31f.jpg  
  inflating: Images/2848266893_9693c66275.jpg  
  inflating: Images/2848571082_26454cb981.jpg  
  inflating: Images/2848895544_6d06210e9d.jpg  
  inflating: Images/2848977044_446a31d86e.jpg  
  inflating: Images/2849194983_2968c72832.jpg  
  inflating: Images/2850719435_221f15e951.jpg  
  inflating: Images/2851198725_37b6027625.jpg  
  inflating: Images/2851304910_b5721199bc.jpg  
  inflating: Images/2851931813_eaf8ed7be3.jpg  
  inflating: Images/2852982055_8112d0

In [ ]:
images_dir = "/content/Images"
captions_path = "/content/captions.txt"

In [ ]:
!pip3 install pickle5

     |████████████████████████████████| 256 kB 4.4 MB/s 


In [ ]:
!pip3 install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-msqz_d8p
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-msqz_d8p
     |████████████████████████████████| 53 kB 1.2 MB/s 
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369221 sha256=d66bd9cc6d813d905a53e76e10e48f0a3fa3bb0d6545e5eb5e4336eb7bce7cee
  Stored in directory: /tmp/pip-ephem-wheel-cache-ooqpahzf/wheels/fd/b9/c3/5b4470e35ed76e174bff77c92f91da82098d5e35fd5bc8cdac
Successfully built clip


In [ ]:
import torch
# check is GPU is available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


## Preprocessing dataset

### test train split

In [ ]:
raw_data = []
caps = []
with open(captions_path, 'r') as f:
    # header
    f.readline()
    line = f.readline()

    index = 1
    while line:

        # if index == 100:
        #   break
        if index % 5 == 1:
            print(index, len(caps))
            if len(caps) != 0:
                print("reset")
                raw_data.append({"img_path": img_path, 
                                "captions": caps})
                caps = []
            
            [img_path, cap] = line.split(sep=",", maxsplit=1) 

            img_path = os.path.join(images_dir, img_path)
            
            # image = io.imread(img_path)
            # image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
            # with torch.no_grad():
            #     prefix = clip_model.encode_image(image).cpu()
            
        else:
            [_, cap] = line.split(sep=",", maxsplit=1) 

        cap = cap.strip()
        cap = cap.strip('\"')
        caps.append(cap)
            # {"caption": cap, "label": 1})

        index += 1

        line = f.readline()

Streaming output truncated to the last 5000 lines.
27956 5
reset
27961 5
reset
27966 5
reset
27971 5
reset
27976 5
reset
27981 5
reset
27986 5
reset
27991 5
reset
27996 5
reset
28001 5
reset
28006 5
reset
28011 5
reset
28016 5
reset
28021 5
reset
28026 5
reset
28031 5
reset
28036 5
reset
28041 5
reset
28046 5
reset
28051 5
reset
28056 5
reset
28061 5
reset
28066 5
reset
28071 5
reset
28076 5
reset
28081 5
reset
28086 5
reset
28091 5
reset
28096 5
reset
28101 5
reset
28106 5
reset
28111 5
reset
28116 5
reset
28121 5
reset
28126 5
reset
28131 5
reset
28136 5
reset
28141 5
reset
28146 5
reset
28151 5
reset
28156 5
reset
28161 5
reset
28166 5
reset
28171 5
reset
28176 5
reset
28181 5
reset
28186 5
reset
28191 5
reset
28196 5
reset
28201 5
reset
28206 5
reset
28211 5
reset
28216 5
reset
28221 5
reset
28226 5
reset
28231 5
reset
28236 5
reset
28241 5
reset
28246 5
reset
28251 5
reset
28256 5
reset
28261 5
reset
28266 5
reset
28271 5
reset
28276 5
reset
28281 5
reset
28286 5
reset
28291 5
res

### start the process

In [ ]:
import torch
import skimage.io as io
import clip
from PIL import Image
import pickle
import json
import os
from tqdm import tqdm
import argparse

clip_model_type = "ViT-B/32"
# def main(clip_model_type: str):
images_dir
# device = torch.device('cuda:0')

clip_model, preprocess = clip.load(clip_model_type, device=device, jit=False)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 72.8MiB/s]


In [ ]:
out_path = f"data.pkl"

In [ ]:
dataset = []

caps = []

with open(captions_path, 'r') as f:
    # header
    f.readline()
    line = f.readline()

    index = 1
    while line:

        # if index == 100:
        #   break
        if index % 5 == 1:
            print(index, len(caps))
            if len(caps) != 0:
                print("reset")
                dataset.append({"embedding": prefix, 
                                "img_path": img_path,
                                "captions": caps})
                caps = []
            
            [img_path, cap] = line.split(sep=",", maxsplit=1) 

            img_path = os.path.join(images_dir, img_path)
            
            image = io.imread(img_path)
            image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
            with torch.no_grad():
                prefix = clip_model.encode_image(image).cpu()
            
        else:
            [_, cap] = line.split(sep=",", maxsplit=1) 

        cap = cap.strip()
        cap = cap.strip('\"')
        caps.append({"caption": cap, "label": 1})

        # all_captions.append(cap)
        # all_embeddings.append(prefix)

        index += 1

        line = f.readline()


Streaming output truncated to the last 5000 lines.
27956 5
reset
27961 5
reset
27966 5
reset
27971 5
reset
27976 5
reset
27981 5
reset
27986 5
reset
27991 5
reset
27996 5
reset
28001 5
reset
28006 5
reset
28011 5
reset
28016 5
reset
28021 5
reset
28026 5
reset
28031 5
reset
28036 5
reset
28041 5
reset
28046 5
reset
28051 5
reset
28056 5
reset
28061 5
reset
28066 5
reset
28071 5
reset
28076 5
reset
28081 5
reset
28086 5
reset
28091 5
reset
28096 5
reset
28101 5
reset
28106 5
reset
28111 5
reset
28116 5
reset
28121 5
reset
28126 5
reset
28131 5
reset
28136 5
reset
28141 5
reset
28146 5
reset
28151 5
reset
28156 5
reset
28161 5
reset
28166 5
reset
28171 5
reset
28176 5
reset
28181 5
reset
28186 5
reset
28191 5
reset
28196 5
reset
28201 5
reset
28206 5
reset
28211 5
reset
28216 5
reset
28221 5
reset
28226 5
reset
28231 5
reset
28236 5
reset
28241 5
reset
28246 5
reset
28251 5
reset
28256 5
reset
28261 5
reset
28266 5
reset
28271 5
reset
28276 5
reset
28281 5
reset
28286 5
reset
28291 5
res

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

data_index = np.array(range(len(dataset)))

train_index, test_index, _, _ = train_test_split(
    data_index, data_index, test_size=0.03, random_state=42)

train_index = train_index.tolist()
test_index = test_index.tolist()
print(len(train_index))
print(len(test_index))

7847
243


In [ ]:
from random import randint
train_dataset = []
test_dataset = []
for i, data in enumerate(dataset):
  if i in train_index:
    train_dataset.append(data)
  else:
    captions = data["captions"]
    j = randint(0, 4)
    data["captions"] = captions[j]
    test_dataset.append(data)

In [ ]:
for i, data in enumerate(test_dataset[:5]):
    print()
    print(i)
    i_captions = data["captions"]
    print(len(i_captions))
    print(i_captions)


0
2
{'caption': 'A man and a little boy in blue life jackets are rowing a yellow canoe .', 'label': 1}

1
2
{'caption': 'A person dressed in a tan jacket jumps a quad over a harvested cornfield in front of a small out building .', 'label': 1}

2
2
{'caption': 'A baby is sitting on and playing with smooth rocks .', 'label': 1}

3
2
{'caption': 'a woman is sitting on a bench with a latte in her lap and a white dog on a blue leash to her side .', 'label': 1}

4
2
{'caption': 'A girl is getting ready to swing the club .', 'label': 1}


In [ ]:
for i, data in enumerate(train_dataset[:5]):
    print()
    print(i)
    i_captions = data["captions"]
    print(len(i_captions))
    print(i_captions)


0
5
[{'caption': 'A child in a pink dress is climbing up a set of stairs in an entry way .', 'label': 1}, {'caption': 'A girl going into a wooden building .', 'label': 1}, {'caption': 'A little girl climbing into a wooden playhouse .', 'label': 1}, {'caption': 'A little girl climbing the stairs to her playhouse .', 'label': 1}, {'caption': 'A little girl in a pink dress going into a wooden cabin .', 'label': 1}]

1
5
[{'caption': 'A black dog and a spotted dog are fighting', 'label': 1}, {'caption': 'A black dog and a tri-colored dog playing with each other on the road .', 'label': 1}, {'caption': 'A black dog and a white dog with brown spots are staring at each other in the street .', 'label': 1}, {'caption': 'Two dogs of different breeds looking at each other on the road .', 'label': 1}, {'caption': 'Two dogs on pavement moving toward each other .', 'label': 1}]

2
5
[{'caption': 'A little girl covered in paint sits in front of a painted rainbow with her hands in a bowl .', 'label':

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def string_dist(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


# print cosine_sim('a little bird', 'a little bird')
# print cosine_sim('a little bird', 'a little bird chirps')
# print cosine_sim('a little bird', 'a big dog barks')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from random import randint
# generate the hard negative sample
for i, data in enumerate(train_dataset):
    print()
    print(i)
    i_captions = data["captions"]

    contrastive_captions = []
    index_tracker = set()

    while len(contrastive_captions) < 6:
        print(len(contrastive_captions))
        j = randint(0, len(train_dataset)-1)

        if j == i:
            continue

        j_captions = train_dataset[j]["captions"]
        j_cap_idx = randint(0, 4)
        j_cap = j_captions[j_cap_idx]["caption"]

        save_cap = True
        for i_cap in i_captions[:5]:
            i_cap = i_cap["caption"]
            sim_score = string_dist(i_cap, j_cap)
            # print(i_cap, j_cap, sim_score)
            if sim_score > 0.05:
                save_cap = False
                break

            if (j, j_cap_idx) in index_tracker:
                save_cap = False
                break

        if save_cap:
            contrastive_captions.append({"caption": j_cap, "label": 0})
            index_tracker.add((j, j_cap_idx))

    data["captions"] += contrastive_captions



0
0
1
2
3
4
5

1
0
0
1
2


/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:401: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'afterward', 'alon', 'alreadi', 'alway', 'ani', 'anoth', 'anyon', 'anyth', 'anywher', 'becam', 'becaus', 'becom', 'befor', 'besid', 'cri', 'describ', 'dure', 'els', 'elsewher', 'empti', 'everi', 'everyon', 'everyth', 'everywher', 'fifti', 'formerli', 'forti', 'ha', 'henc', 'hereaft', 'herebi', 'hi', 'howev', 'hundr', 'inde', 'latterli', 'mani', 'meanwhil', 'moreov', 'mostli', 'nobodi', 'noon', 'noth', 'nowher', 'onc', 'onli', 'otherwis', 'ourselv', 'perhap', 'pleas', 'seriou', 'sever', 'sinc', 'sincer', 'sixti', 'someon', 'someth', 'sometim', 'somewher', 'themselv', 'thenc', 'thereaft', 'therebi', 'therefor', 'thi', 'thu', 'togeth', 'twelv', 'twenti', 'veri', 'wa', 'whatev', 'whenc', 'whenev', 'wherea', 'whereaft', 'wherebi', 'wherev', 'whi', 'yourselv'] not in stop_words.
  % sorted(

Streaming output truncated to the last 5000 lines.
2
3
3
4
5
5

7423
0
1
2
3
4
5

7424
0
0
0
0
1
2
3
3
3
4
5

7425
0
1
2
2
3
4
5
5

7426
0
0
1
2
3
3
4
5

7427
0
0
0
1
2
3
4
4
4
5

7428
0
1
1
1
2
3
3
3
4
5

7429
0
0
1
1
1
2
3
4
5
5

7430
0
0
1
1
2
2
3
3
3
3
4
4
5

7431
0
0
1
1
1
1
2
3
4
5

7432
0
0
1
1
1
1
1
1
1
2
3
4
4
4
5

7433
0
1
2
3
3
4
5

7434
0
1
2
2
3
3
3
4
5

7435
0
0
1
2
2
3
4
4
4
4
5
5

7436
0
1
2
3
4
4
5

7437
0
1
2
3
4
5

7438
0
1
2
3
4
4
5
5

7439
0
1
1
2
3
4
4
4
4
5

7440
0
1
2
3
4
4
5

7441
0
0
0
0
0
0
0
1
1
1
2
2
3
3
4
5

7442
0
0
0
1
1
2
2
2
2
2
3
4
5
5

7443
0
1
2
3
4
5

7444
0
1
2
3
4
5

7445
0
1
2
3
4
4
4
5
5
5
5

7446
0
1
2
3
4
5

7447
0
1
1
1
2
3
3
4
5
5

7448
0
0
0
1
2
3
4
5
5

7449
0
0
1
1
2
3
4
5

7450
0
0
0
1
2
3
4
4
5
5

7451
0
1
1
2
3
3
4
4
5

7452
0
1
2
3
3
4
5

7453
0
1
1
2
3
3
4
4
4
5
5
5
5
5

7454
0
0
1
1
2
3
4
5

7455
0
1
2
3
3
4
5

7456
0
0
1
2
3
4
4
5

7457
0
1
2
2
2
2
2
3
4
5
5

7458
0
1
1
2
3
3
4
5

7459
0
1
2
3
3
3
3
4
5
5

7460
0
1
1
2
3
4
5

7461

In [ ]:
len(train_dataset)

7847

In [ ]:
train_dataset[0]["captions"]

[{'caption': 'A child in a pink dress is climbing up a set of stairs in an entry way .',
  'label': 1},
 {'caption': 'A girl going into a wooden building .', 'label': 1},
 {'caption': 'A little girl climbing into a wooden playhouse .', 'label': 1},
 {'caption': 'A little girl climbing the stairs to her playhouse .',
  'label': 1},
 {'caption': 'A little girl in a pink dress going into a wooden cabin .',
  'label': 1},
 {'caption': 'a brown dog jumping off the ground in the woods', 'label': 0},
 {'caption': 'A man with four running dogs in nature .', 'label': 0},
 {'caption': 'A man in red winter clothes talks to another man in red .',
  'label': 0},
 {'caption': 'A woman in a skirt and green t-shirt has her arm draped over the shoulder of a younger woman in a white skirt and orange t-shirt .',
  'label': 0},
 {'caption': 'The grey and white dog is running through the snow .',
  'label': 0},
 {'caption': 'A man and a woman sit back-to-back on a bench outside a storefront .',
  'label': 

In [ ]:
# save to pickle
import pickle5 as pickle
with open('train_'+out_path, 'wb') as f:
    pickle.dump(train_dataset, f)
with open('test_'+out_path, 'wb') as f:
    pickle.dump(test_dataset, f)

print('Done')
print("%0d embeddings saved " % len(dataset))

Done
8090 embeddings saved 



### Investigate text similarity method

In [ ]:
from gensim import corpora
import gensim.downloader as api
from gensim.utils import simple_preprocess
from gensim.matutils import softcossim

glove = api.load("glove-wiki-gigaword-50")

In [ ]:

# this is not acceptable

# 'A dog in a field .', 'A dog is resting in a grassy field', 'A dog runs through tall grass .', 'Dog running after another dog in a grassy field .'

corpus = ["A dog in a field .", 'A dog runs through tall grass .']
# corpus = ["A dog in a field .", "Four people are lining up to purchase tickets at the theater ."]
# corpus = ["A girl skiing .", "Four people are lining up to purchase tickets at the theater ."]
# corpus = ['a brown dog is jumping up at a woman who is wearing a brown skirt .','A golden retriever is excitedly greeting a woman as a little dog walks towards the camera .']
dictionary = corpora.Dictionary([simple_preprocess(doc) for doc in corpus])
sim_matrix = glove.similarity_matrix(dictionary=dictionary)
sent_1 = dictionary.doc2bow(simple_preprocess(corpus[0]))
sent_2 = dictionary.doc2bow(simple_preprocess(corpus[1]))
print(f"Soft Cosine Similarity: {softcossim(sent_1, sent_2, sim_matrix)}")


Soft Cosine Similarity: 0.7053386207502227


In [ ]:
cosine_sim(corpus[0],corpus[1])

0.22028815056182965

In [ ]:
import nltk, string
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt') # if necessary...


stemmer = nltk.stem.porter.PorterStemmer()
remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)

def stem_tokens(tokens):
    return [stemmer.stem(item) for item in tokens]

'''remove punctuation, lowercase, stem'''
def normalize(text):
    return stem_tokens(nltk.word_tokenize(text.lower().translate(remove_punctuation_map)))

vectorizer = TfidfVectorizer(tokenizer=normalize, stop_words='english')

def string_dist(text1, text2):
    tfidf = vectorizer.fit_transform([text1, text2])
    return ((tfidf * tfidf.T).A)[0,1]


# print cosine_sim('a little bird', 'a little bird')
# print cosine_sim('a little bird', 'a little bird chirps')
# print cosine_sim('a little bird', 'a big dog barks')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:

from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
import sys

class ClipDataset(Dataset):

    def __len__(self) -> int:
        return len(self.captions_tokens)

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            self.captions_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        tokens[~mask] = 0
        mask = mask.float()
        mask = torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask

    def __getitem__(self, item: int):
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[self.caption2embedding[item]]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        return tokens, mask, prefix

    def __init__(self, data_path: str,  prefix_length: int, gpt2_type: str = "gpt2",
                 normalize_prefix=False):
        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)
        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            all_data = pickle.load(f)
        print("Data size is %0d" % len(all_data))
        sys.stdout.flush()
        self.prefixes = all_data["clip_embedding"]
        captions_raw = all_data["captions"]
        self.image_ids = [caption["image_id"] for caption in captions_raw]
        self.captions = [caption['caption'] for caption in captions_raw]
        if os.path.isfile(f"{data_path[:-4]}_tokens.pkl"):
            with open(f"{data_path[:-4]}_tokens.pkl", 'rb') as f:
                self.captions_tokens, self.caption2embedding, self.max_seq_len = pickle.load(f)
        else:
            self.captions_tokens = []
            self.caption2embedding = []
            max_seq_len = 0
            for caption in captions_raw:
                self.captions_tokens.append(torch.tensor(self.tokenizer.encode(caption['caption']), dtype=torch.int64))
                self.caption2embedding.append(caption["clip_embedding"])
                max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])
            # self.max_seq_len = max_seq_len
            with open(f"{data_path[:-4]}_tokens.pkl", 'wb') as f:
                pickle.dump([self.captions_tokens, self.caption2embedding, max_seq_len], f)
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))



## Loading CLIP model and encoding image

In [ ]:
filename = "667626_18933d713e.jpg"
clip_model_type = "ViT-B/32"

In [ ]:
# load clip model
import clip
import skimage.io as io
from PIL import Image

clip_model, preprocess = clip.load(clip_model_type, device=device, jit=False)

100%|███████████████████████████████████████| 338M/338M [00:04<00:00, 86.3MiB/s]


In [ ]:
image = io.imread(filename)
image = preprocess(Image.fromarray(image)).unsqueeze(0).to(device)
with torch.no_grad():
    prefix = clip_model.encode_image(image).cpu()

In [ ]:
prefix.size()

torch.Size([1, 512])

## Loading GPT-2 for sequence classification

In [ ]:
!pip3 install transformers

     |████████████████████████████████| 4.0 MB 7.5 MB/s 
     |████████████████████████████████| 596 kB 33.0 MB/s 
     |████████████████████████████████| 6.5 MB 45.9 MB/s 
     |████████████████████████████████| 895 kB 34.1 MB/s 
     |████████████████████████████████| 77 kB 5.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
from transformers import GPT2Tokenizer, GPT2ForSequenceClassification
import torch

tokenizer = GPT2Tokenizer.from_pretrained('microsoft/DialogRPT-updown')
model = GPT2ForSequenceClassification.from_pretrained('microsoft/DialogRPT-updown', return_dict=True)



Downloading:   0%|          | 0.00/0.99M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/31.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/812 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.42G [00:00<?, ?B/s]

In [ ]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 1024)
    (wpe): Embedding(1024, 1024)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D()
          (c_proj): Conv1D()
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (1): GPT2Block(
        (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D()
          (c_proj): Conv1D()
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (

In [ ]:
text = "Hello, my dog is cute [SEP]"

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("microsoft/DialogRPT-updown")

In [ ]:
# tokenizer.add_special_tokens(["[SEP]", "[EOS]"])
tokenizer.add_tokens(['[EOT]'], special_tokens=True)
tokenizer.add_tokens(['[SEP]'], special_tokens=True)

1

In [ ]:
print(len(tokenizer))
tokens = []

tokens.append(tokenizer.encode(["Hello, my dog is cute", "He is really nice"]))

tokens.append(tokenizer.encode("Hello, my dog is cute", "He is really nice"))

tokens.append(tokenizer.encode(["Hello, my dog is cute"]))

tokens.append(tokenizer.encode("Hello, my dog is cute.[SEP]"))

for t in tokens:
    print(tokenizer.convert_ids_to_tokens(t))

50259
['<|endoftext|>', '<|endoftext|>']
['Hello', ',', 'Ġmy', 'Ġdog', 'Ġis', 'Ġcute', 'He', 'Ġis', 'Ġreally', 'Ġnice']
['<|endoftext|>']
['Hello', ',', 'Ġmy', 'Ġdog', 'Ġis', 'Ġcute', '.', '[SEP]']


In [ ]:
tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

In [ ]:
tokenizer(text)

{'input_ids': [15496, 11, 616, 3290, 318, 13779, 685, 5188, 47, 60], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [ ]:
from transformers import BertTokenizer, BertForNextSentencePrediction

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True, use_fast=True)

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
print(len(tokenizer))
tokens = []

tokens.append(tokenizer.encode(["Hello, my dog is cute", "He is really nice"]))

tokens.append(tokenizer.encode("Hello, my dog is cute", "He is really nice"))

tokens.append(tokenizer.encode(["Hello, my dog is cute"]))

tokens.append(tokenizer.encode("Hello, my dog is cute"))

for t in tokens:
    print(tokenizer.convert_ids_to_tokens(t))

30522
['[CLS]', '[UNK]', '[UNK]', '[SEP]']
['[CLS]', 'hello', ',', 'my', 'dog', 'is', 'cute', '[SEP]', 'he', 'is', 'really', 'nice', '[SEP]']
['[CLS]', '[UNK]', '[SEP]']
['[CLS]', 'hello', ',', 'my', 'dog', 'is', 'cute', '[SEP]']


In [ ]:

inputs = tokenizer(text, return_tensors="pt")
labels = torch.tensor([1]).unsqueeze(0)  # Batch size 1
outputs = model(**inputs, labels=labels)
loss = outputs.loss
logits = outputs.logits

In [ ]:
inputs

{'input_ids': tensor([[15496,    11,   616,  3797,   318, 13779]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [ ]:
tokens = torch.tensor(tokenizer.encode(text), dtype=torch.int64)
model.gpt.transformer.wte(tokens).size()

torch.Size([6, 1024])

In [ ]:
from torch import nn

print(logits)
print(nn.functional.softmax(logits, dim=1))

tensor([[-1.2862]], grad_fn=<IndexBackward0>)


tensor([[1.]], grad_fn=<SoftmaxBackward0>)

In [ ]:
class MLP(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    # sizes here is a tuple
    def __init__(self, sizes, bias=True, act=nn.Tanh):
        print(sizes)
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)

NameError: ignored

In [ ]:
net = MLP((512, 5120, 10240))
x = torch.randn(1,512)
y = net(x)
print(y.size())
print(net)

(512, 5120, 10240)
torch.Size([1, 10240])
MLP(
  (model): Sequential(
    (0): Linear(in_features=512, out_features=5120, bias=True)
    (1): Tanh()
    (2): Linear(in_features=5120, out_features=10240, bias=True)
  )
)


In [ ]:
print(y.view(-1, 10, 1024).size())

torch.Size([1, 10, 1024])


In [ ]:
from typing import Optional
class ImageSequenceClassificationModel(nn.Module):

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        if labels is not None:
            dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
            labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, clip_length: Optional[int] = None, prefix_size: int = 512,
                 num_layers: int = 8):
                #  mapping_type: MappingType = MappingType.MLP):
        super(ImageSequenceClassificationModel, self).__init__()
        self.prefix_length = prefix_length
        self.gpt = GPT2ForSequenceClassification.from_pretrained('microsoft/DialogRPT-updown', return_dict=True)
        # GPT2LMHeadModel.from_pretrained('gpt2')

        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]
        # if mapping_type == MappingType.MLP:
        print(self.gpt_embedding_size)
        print(prefix_length)
    
        self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                  self.gpt_embedding_size * prefix_length))
        # else:
        #     self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, prefix_length,
        #                                                              clip_length, num_layers)


In [ ]:

prefix_length = 10
prefix_length_clip = 10
num_layers = 8

prefix_dim = 512
# 640 if args.is_rn else 512

model = ImageSequenceClassificationModel(prefix_length, clip_length=prefix_length_clip, prefix_size=prefix_dim,
                              num_layers=num_layers)

    

1024
10
(512, 5120, 10240)


In [ ]:
model

ImageSequenceClassificationModel(
  (gpt): GPT2ForSequenceClassification(
    (transformer): GPT2Model(
      (wte): Embedding(50257, 1024)
      (wpe): Embedding(1024, 1024)
      (drop): Dropout(p=0.1, inplace=False)
      (h): ModuleList(
        (0): GPT2Block(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
            (c_proj): Conv1D()
            (attn_dropout): Dropout(p=0.1, inplace=False)
            (resid_dropout): Dropout(p=0.1, inplace=False)
          )
          (ln_2): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (mlp): GPT2MLP(
            (c_fc): Conv1D()
            (c_proj): Conv1D()
            (act): NewGELUActivation()
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (1): GPT2Block(
          (ln_1): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
          (attn): GPT2Attention(
            (c_attn): Conv1D()
   

In [ ]:
def train(dataset, model, lr: float = 2e-5, warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):

    device = torch.device('cuda:0')
    batch_size = args.bs
    epochs = args.epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = model.to(device)
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs * len(train_dataloader)
    )
    # save_config(args)
    for epoch in range(epochs):
        print(f">>> Training epoch {epoch}")
        sys.stdout.flush()
        progress = tqdm(total=len(train_dataloader), desc=output_prefix)
        for idx, (tokens, mask, prefix) in enumerate(train_dataloader):
            model.zero_grad()
            tokens, mask, prefix = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32)
            outputs = model(tokens, prefix, mask)
            logits = outputs.logits[:, dataset.prefix_length - 1: -1]
            loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)
            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            progress.set_postfix({"loss": loss.item()})
            progress.update()
            if (idx + 1) % 10000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        progress.close()
        if epoch % args.save_every == 0 or epoch == epochs - 1:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch:03d}.pt"),
            )
    return model


## Start train.py process

In [ ]:
!pip install wandb



In [ ]:
import wandb

wandb.login()


wandb: Currently logged in as: geekhusky (use `wandb login --relogin` to force relogin)


True

In [ ]:
wandb.init(project="encoder_seq_classification")

In [ ]:
!pip3 install transformers

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as nnf
from torch.utils.data import Dataset, DataLoader
from enum import Enum

from transformers import GPT2Config, GPT2Tokenizer, AdamW, get_linear_schedule_with_warmup, GPT2ForSequenceClassification
import torch


import os
import pickle
import sys
import argparse
import json
from typing import Tuple, Optional, Union


import logging
from tqdm import tqdm

class MappingType(Enum):
    MLP = 'mlp'
    Transformer = 'transformer'


class ClipCocoDataset(Dataset):

    def __len__(self) -> int:
        return len(self.captions_tokens)

    def pad_tokens(self, item: int):
        tokens = self.captions_tokens[item]
        padding = self.max_seq_len - tokens.shape[0]
        if padding > 0:
            # padding to the left
            # tokens = torch.cat((tokens, torch.zeros(padding, dtype=torch.int64) - 1))
            tokens = torch.cat((torch.zeros(padding, dtype=torch.int64) - 1, tokens))
            # self.captions_tokens[item] = tokens
        elif padding < 0:
            tokens = tokens[:self.max_seq_len]
            # self.captions_tokens[item] = tokens
        mask = tokens.ge(0)  # mask is zero where we out of sequence
        # print(mask)
        # print(tokens)
        tokens[~mask] = 0
        # print(tokens)
        mask = mask.float()

        # +1 for the tokenized_EOT
        mask = torch.cat((mask, torch.ones(self.prefix_length+1)), dim=0)
        # mask = torch.cat((mask, torch.ones(self.prefix_length)), dim=0)
        # torch.cat((torch.ones(self.prefix_length), mask), dim=0)  # adding prefix mask
        return tokens, mask

    def __getitem__(self, item: int) -> Tuple[torch.Tensor, ...]:
        tokens, mask = self.pad_tokens(item)
        prefix = self.prefixes[self.caption2embedding[item]]
        if self.normalize_prefix:
            prefix = prefix.float()
            prefix = prefix / prefix.norm(2, -1)
        label = self.labels[item]
        return tokens, mask, prefix, label

    def __init__(self, data_path: str,  prefix_length: int, gpt2_type: str = "gpt2",
                 normalize_prefix=False, test=False):

        self.tokenizer = GPT2Tokenizer.from_pretrained(gpt2_type)

        self.tokenizer.add_tokens(['[BOT]'], special_tokens=True)
        self.tokenizer.add_tokens(['[SEP]'], special_tokens=True)
        self.tokenizer.add_tokens(['[EOT]'], special_tokens=True)

        self.tokenized_EOT = torch.tensor(self.tokenizer.encode('[EOT]'), dtype=torch.int64)

        self.prefix_length = prefix_length
        self.normalize_prefix = normalize_prefix
        with open(data_path, 'rb') as f:
            dataset = pickle.load(f)
        print("Data size is %0d" % len(dataset))
        sys.stdout.flush()
        self.prefixes = []
        self.captions = []
        self.labels = []
        for data in dataset:
            captions = data["captions"]
            self.prefixes += [data["embedding"]]*len(captions)
            
            if not test:
              for cap in captions:
                  # self.labels.append(torch.tensor([cap["label"]]).unsqueeze(0))
                  if cap["label"] == 0:
                    self.labels.append(torch.tensor([1,0]))
                  else:
                    self.labels.append(torch.tensor([0,1]))
                  self.captions.append(cap["caption"])
            else:
              self.captions.append(captions["caption"])
              self.labels.append(torch.tensor([0,1]))

        # self.image_ids = [caption["image_id"] for caption in captions_raw]
        # self.captions = [caption['caption'] for caption in captions_raw]
        if os.path.isfile(f"{data_path[:-4]}_tokens.pkl"):
            with open(f"{data_path[:-4]}_tokens.pkl", 'rb') as f:
                self.captions_tokens, self.caption2embedding, self.max_seq_len = pickle.load(f)
        else:
            self.captions_tokens = []
            self.caption2embedding = []
            max_seq_len = 0
            for i, caption in enumerate(self.captions):
                self.captions_tokens.append(torch.tensor(self.tokenizer.encode('[BOT]'+caption+'[SEP]'), dtype=torch.int64))
                self.caption2embedding.append(i)
                max_seq_len = max(max_seq_len, self.captions_tokens[-1].shape[0])
            # self.max_seq_len = max_seq_len
            with open(f"{data_path[:-4]}_tokens.pkl", 'wb') as f:
                pickle.dump([self.captions_tokens, self.caption2embedding, max_seq_len], f)
        all_len = torch.tensor([len(self.captions_tokens[i]) for i in range(len(self))]).float()
        self.max_seq_len = min(int(all_len.mean() + all_len.std() * 10), int(all_len.max()))


class MLP(nn.Module):

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        return self.model(x)

    def __init__(self, sizes: Tuple[int, ...], bias=True, act=nn.Tanh):
        super(MLP, self).__init__()
        layers = []
        for i in range(len(sizes) - 1):
            layers.append(nn.Linear(sizes[i], sizes[i + 1], bias=bias))
            if i < len(sizes) - 2:
                layers.append(act())
        self.model = nn.Sequential(*layers)


class MlpTransformer(nn.Module):
    def __init__(self, in_dim, h_dim, out_d: Optional[int] = None, act=nnf.relu, dropout=0.):
        super().__init__()
        out_d = out_d if out_d is not None else in_dim
        self.fc1 = nn.Linear(in_dim, h_dim)
        self.act = act
        self.fc2 = nn.Linear(h_dim, out_d)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = self.fc1(x)
        x = self.act(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

class MultiHeadAttention(nn.Module):

    def __init__(self, dim_self, dim_ref, num_heads, bias=True, dropout=0.):
        super().__init__()
        self.num_heads = num_heads
        head_dim = dim_self // num_heads
        self.scale = head_dim ** -0.5
        self.to_queries = nn.Linear(dim_self, dim_self, bias=bias)
        self.to_keys_values = nn.Linear(dim_ref, dim_self * 2, bias=bias)
        self.project = nn.Linear(dim_self, dim_self)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y=None, mask=None):
        y = y if y is not None else x
        b, n, c = x.shape
        _, m, d = y.shape
        # b n h dh
        queries = self.to_queries(x).reshape(b, n, self.num_heads, c // self.num_heads)
        # b m 2 h dh
        keys_values = self.to_keys_values(y).reshape(b, m, 2, self.num_heads, c // self.num_heads)
        keys, values = keys_values[:, :, 0], keys_values[:, :, 1]
        attention = torch.einsum('bnhd,bmhd->bnmh', queries, keys) * self.scale
        if mask is not None:
            if mask.dim() == 2:
                mask = mask.unsqueeze(1)
            attention = attention.masked_fill(mask.unsqueeze(3), float("-inf"))
        attention = attention.softmax(dim=2)
        out = torch.einsum('bnmh,bmhd->bnhd', attention, values).reshape(b, n, c)
        out = self.project(out)
        return out, attention


class TransformerLayer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        x_, attention = self.attn(self.norm1(x), y, mask)
        x = x + x_
        x = x + self.mlp(self.norm2(x))
        return x, attention

    def forward(self, x, y=None, mask=None):
        x = x + self.attn(self.norm1(x), y, mask)[0]
        x = x + self.mlp(self.norm2(x))
        return x

    def __init__(self, dim_self, dim_ref, num_heads, mlp_ratio=4., bias=False, dropout=0., act=nnf.relu,
                 norm_layer: nn.Module = nn.LayerNorm):
        super().__init__()
        self.norm1 = norm_layer(dim_self)
        self.attn = MultiHeadAttention(dim_self, dim_ref, num_heads, bias=bias, dropout=dropout)
        self.norm2 = norm_layer(dim_self)
        self.mlp = MlpTransformer(dim_self, int(dim_self * mlp_ratio), act=act, dropout=dropout)


class Transformer(nn.Module):

    def forward_with_attention(self, x, y=None, mask=None):
        attentions = []
        for layer in self.layers:
            x, att = layer.forward_with_attention(x, y, mask)
            attentions.append(att)
        return x, attentions

    def forward(self, x, y=None, mask=None):
        for i, layer in enumerate(self.layers):
            if i % 2 == 0 and self.enc_dec: # cross
                x = layer(x, y)
            elif self.enc_dec:  # self
                x = layer(x, x, mask)
            else:  # self or cross
                x = layer(x, y, mask)
        return x

    def __init__(self, dim_self: int, num_heads: int, num_layers: int, dim_ref: Optional[int] = None,
                 mlp_ratio: float = 2., act=nnf.relu, norm_layer: nn.Module = nn.LayerNorm, enc_dec: bool = False):
        super(Transformer, self).__init__()
        dim_ref = dim_ref if dim_ref is not None else dim_self
        self.enc_dec = enc_dec
        if enc_dec:
            num_layers = num_layers * 2
        layers = []
        for i in range(num_layers):
            if i % 2 == 0 and enc_dec:  # cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            elif enc_dec:  # self
                layers.append(TransformerLayer(dim_self, dim_self, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
            else:  # self or cross
                layers.append(TransformerLayer(dim_self, dim_ref, num_heads, mlp_ratio, act=act, norm_layer=norm_layer))
        self.layers = nn.ModuleList(layers)


class TransformerMapper(nn.Module):

    def forward(self, x):
        x = self.linear(x).view(x.shape[0], self.clip_length, -1)
        prefix = self.prefix_const.unsqueeze(0).expand(x.shape[0], *self.prefix_const.shape)
        prefix = torch.cat((x, prefix), dim=1)
        out = self.transformer(prefix)[:, self.clip_length:]
        return out

    def __init__(self, dim_clip: int, dim_embedding: int, prefix_length: int, clip_length: int, num_layers: int = 8):
        super(TransformerMapper, self).__init__()
        self.clip_length = clip_length
        self.transformer = Transformer(dim_embedding, 8, num_layers)
        self.linear = nn.Linear(dim_clip, clip_length * dim_embedding)
        self.prefix_const = nn.Parameter(torch.randn(prefix_length, dim_embedding), requires_grad=True)


class ClipCaptionModel(nn.Module):

    def get_dummy_token(self, batch_size: int, device: torch.device) -> torch.Tensor:
        return torch.zeros(batch_size, self.prefix_length, dtype=torch.int64, device=device)

    def forward(self, tokens: torch.Tensor, prefix: torch.Tensor, mask: Optional[torch.Tensor] = None,
                labels: Optional[torch.Tensor] = None):
        embedding_text = self.gpt.transformer.wte(tokens)
        # print("emb text:", embedding_text.size())
        embedding_EOT = self.gpt.transformer.wte(self.tokenized_EOT.repeat(tokens.size(0), 1))
        # print("emb eot:", embedding_EOT.size())
        prefix_projections = self.clip_project(prefix).view(-1, self.prefix_length, self.gpt_embedding_size)
        # print("prefix:", prefix_projections.size())

        embedding_cat = torch.cat((embedding_text, prefix_projections, embedding_EOT), dim=1)

        # print(embedding_cat.size())
        # print(mask.size())
        # embedding_cat = torch.cat((prefix_projections, embedding_text), dim=1)
        # print(embedding_text.size())
        # print(prefix_projections.size())
        # print(embedding_cat.size())
        # if labels is not None:
        #     dummy_token = self.get_dummy_token(tokens.shape[0], tokens.device)
        #     labels = torch.cat((dummy_token, tokens), dim=1)
        out = self.gpt(inputs_embeds=embedding_cat, labels=labels, attention_mask=mask)
        return out

    def __init__(self, prefix_length: int, clip_length: Optional[int] = None, prefix_size: int = 512,
                 num_layers: int = 8, mapping_type: MappingType = MappingType.MLP, tokenizer_length = 0,
                 tokenized_EOT=None):
      
        super(ClipCaptionModel, self).__init__()
        self.prefix_length = prefix_length

        self.tokenized_EOT = tokenized_EOT
      
        model_config = GPT2Config.from_pretrained(pretrained_model_name_or_path="gpt2", num_labels=2)
        self.gpt = GPT2ForSequenceClassification.from_pretrained(pretrained_model_name_or_path="gpt2", config=model_config)
        # GPT2ForSequenceClassification.from_pretrained('microsoft/DialogRPT-updown', return_dict=True,\
                                                                  # problem_type="multi_label_classification")

        self.gpt.resize_token_embeddings(tokenizer_length)
        self.gpt.config.pad_token_id = 0
        # GPT2LMHeadModel.from_pretrained('gpt2')
        
        self.gpt_embedding_size = self.gpt.transformer.wte.weight.shape[1]

        if mapping_type == MappingType.MLP:
            self.clip_project = MLP((prefix_size, (self.gpt_embedding_size * prefix_length) // 2,
                                     self.gpt_embedding_size * prefix_length))
        else:
            self.clip_project = TransformerMapper(prefix_size, self.gpt_embedding_size, prefix_length,
                                                                     clip_length, num_layers)


class ClipCaptionPrefix(ClipCaptionModel):

    def parameters(self, recurse: bool = True):
        return self.clip_project.parameters()

    def train(self, mode: bool = True):
        super(ClipCaptionPrefix, self).train(mode)
        self.gpt.eval()
        return self


def save_config(args: argparse.Namespace):
    config = {}
    for key, item in args._get_kwargs():
        config[key] = item
    out_path = os.path.join(args.out_dir, f"{args.prefix}.json")
    with open(out_path, 'w') as outfile:
        json.dump(config, outfile)


def load_model(config_path: str, epoch_or_latest: Union[str, int] = '_latest'):
    with open(config_path) as f:
        config = json.load(f)
    parser = argparse.ArgumentParser()
    parser.set_defaults(**config)
    args = parser.parse_args()
    if type(epoch_or_latest) is int:
        epoch_or_latest = f"-{epoch_or_latest:03d}"
    model_path = os.path.join(args.out_dir, f"{args.prefix}{epoch_or_latest}.pt")
    if args.only_prefix:
        model = ClipCaptionPrefix(args.prefix_length)
    else:
        model = ClipCaptionModel(args.prefix_length)
    if os.path.isfile(model_path):
        print(f"loading model from {model_path}")
        model.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))
    else:
        print(f"{model_path} is not exist")
    return model, parser


def train(dataset: ClipCocoDataset, model: ClipCaptionModel,  epochs, save_every, bs,
          lr: float = 2e-5, warmup_steps: int = 5000, output_dir: str = ".", output_prefix: str = ""):


    logging.getLogger("transformers").setLevel(logging.ERROR)

    # device = torch.device('cuda:0')
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    print(device)

    batch_size = bs
    epochs = epochs
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    model = model.to(device)
    model.train()
    optimizer = AdamW(model.parameters(), lr=lr)
    train_dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
    scheduler = get_linear_schedule_with_warmup(
        optimizer, num_warmup_steps=warmup_steps, num_training_steps=epochs * len(train_dataloader)
    )
    # save_config(args)
    for epoch in range(epochs):
        # print(f">>> Training epoch {epoch}")
        # sys.stdout.flush()
        # progress = tqdm(total=len(train_dataloader), desc=output_prefix)


        epoch_iterator = tqdm(train_dataloader)
        tr_loss = 0
        correct = 0
        total = 0


        for idx, (tokens, mask, prefix, label) in enumerate(epoch_iterator):

        # for idx, (tokens, mask, prefix, label) in enumerate(train_dataloader):
  
            model.zero_grad()
            tokens, mask, prefix, label = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32), \
                                          label.to(device, dtype=torch.float32)
            outputs = model(tokens, prefix, mask, label)
            # logits = outputs.logits[:, dataset.prefix_length - 1: -1]
            # loss = nnf.cross_entropy(logits.reshape(-1, logits.shape[-1]), tokens.flatten(), ignore_index=0)
            
            loss, logits = outputs[:2]

            loss.backward()
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()
            # progress.set_postfix({"loss": loss.item()})
            # progress.update()

            predictions = torch.argmax(logits, dim = 1)
    
            cur_correct = predictions.eq(torch.argmax(label, dim = 1)).sum().item()

            correct += cur_correct
            total += label.size(0)
            # print(predictions)
            # print(torch.argmax(label, dim = 1))
            # print(cur_correct)
            # print(label.size(0))



            tr_loss += loss.item()
            epoch_iterator.set_description(f'Epoch {epoch}')
            epoch_iterator.set_postfix(loss=tr_loss/(idx+1), accuracy=100*(correct/total))

            metrics = {"train/train_loss": tr_loss/(idx+1), "train/train_acc": 100*(correct/total)}
            wandb.log(metrics)

            if (idx + 1) % 10000 == 0:
                torch.save(
                    model.state_dict(),
                    os.path.join(output_dir, f"{output_prefix}_latest.pt"),
                )
        # progress.close()
        if epoch % save_every == 0 or epoch == epochs - 1:
            torch.save(
                model.state_dict(),
                os.path.join(output_dir, f"{output_prefix}-{epoch:03d}.pt"),
            )
    return model

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/stat940/stat940_final_project/train_data.pkl"
# './data/flickr/ViT-B_32_train.pkl'
out_dir = './checkpoints'
prefix = 'flickr_prefix'
epochs = 10
save_every = 1
prefix_length = 10
prefix_length_clip = 10
bs = 15
only_prefix = False
mapping_type_name = 'transformer'
# 'mlp'
num_layers = 8
is_rn = False
normalize_prefix = False

prefix_length = prefix_length

In [ ]:
# from pickle5.pickle import FALSE
# def main():
# parser = argparse.ArgumentParser()
# parser.add_argument('--data', default='./data/flickr/ViT-B_32_train.pkl')
# parser.add_argument('--out_dir', default='./checkpoints')
# parser.add_argument('--prefix', default='coco_prefix', help='prefix for saved filenames')
# parser.add_argument('--epochs', type=int, default=10)
# parser.add_argument('--save_every', type=int, default=1)
# parser.add_argument('--prefix_length', type=int, default=10)
# parser.add_argument('--prefix_length_clip', type=int, default=10)
# parser.add_argument('--bs', type=int, default=40)
# parser.add_argument('--only_prefix', dest='only_prefix', action='store_true')
# parser.add_argument('--mapping_type', type=str, default='mlp', help='mlp/transformer')
# parser.add_argument('--num_layers', type=int, default=8)
# parser.add_argument('--is_rn', dest='is_rn', action='store_true')
# parser.add_argument('--normalize_prefix', dest='normalize_prefix', action='store_true')


dataset = ClipCocoDataset(data_path, prefix_length, 'gpt2', normalize_prefix=normalize_prefix)
prefix_dim = 640 if is_rn else 512
mapping_type = {'mlp': MappingType.MLP, 'transformer': MappingType.Transformer}[mapping_type_name]
if only_prefix:
    model = ClipCaptionPrefix(prefix_length, clip_length=prefix_length_clip, prefix_size=prefix_dim,
                              num_layers=num_layers, mapping_type=mapping_type, tokenizer_length=len(dataset.tokenizer),
                              tokenized_EOT=dataset.tokenized_EOT)
    print("Train only prefix")
else:
    model = ClipCaptionModel(prefix_length, clip_length=prefix_length_clip, prefix_size=prefix_dim,
                              num_layers=num_layers, mapping_type=mapping_type, tokenizer_length=len(dataset.tokenizer),
                              tokenized_EOT=dataset.tokenized_EOT)
    print("Train both prefix and GPT")



Data size is 7847


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train both prefix and GPT


In [ ]:
dataset.tokenized_EOT

tensor([50259])

In [ ]:
index = 0
for data in dataset:
  if index == 7:
    break
  tokens = data[0]
  mask = data[1]
  label = data[-1]
  print("token len",len(tokens))
  print("mask len",len(mask))
  print(mask)
  print(label)
  print(tokens)

  # print(label.size())

  index += 1

token len 43
mask len 54
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 1])
tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0, 50257,    32,  1200,   287,   257, 11398,  6576,
          318, 14281,   510,   257,   900,   286, 16046,   287,   281,  5726,
          835,   764, 50258])
token len 43
mask len 54
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.])
tensor([0, 1])
tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,    

In [ ]:
train(dataset, model,  epochs, save_every, bs, output_dir=out_dir, output_prefix=prefix)

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


cpu


  0%|          | 0/5932 [00:00<?, ?it/s]

emb text: torch.Size([15, 43, 768])
emb eot: torch.Size([15, 1, 768])
prefix: torch.Size([15, 10, 768])
torch.Size([15, 54, 768])
torch.Size([15, 54])


Epoch 0:   0%|          | 0/5932 [00:11<?, ?it/s, accuracy=46.7, loss=1.92]


Error: ignored

In [ ]:
!nvidia-smi

Sat Apr  9 15:34:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   50C    P0    27W /  70W |  14962MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluation

#### Load model

In [ ]:
wandb.restore('checkpoints/flickr_prefix-006.pt', run_path="geekhusky/encoder_seq_classification/kjb3mmj7")


NameError: ignored

In [ ]:
model_path = "wandb/latest-run/files/checkpoints/flickr_prefix-006.pt"

In [ ]:
data_path = "/content/drive/MyDrive/Colab Notebooks/stat940/stat940_final_project/test_data.pkl"
test_dataset = ClipCocoDataset(data_path, prefix_length, 'gpt2', normalize_prefix=normalize_prefix, test=True)
prefix_dim = 640 if is_rn else 512
mapping_type = {'mlp': MappingType.MLP, 'transformer': MappingType.Transformer}[mapping_type_name]
if only_prefix:
    model = ClipCaptionPrefix(prefix_length, clip_length=prefix_length_clip, prefix_size=prefix_dim,
                              num_layers=num_layers, mapping_type=mapping_type, tokenizer_length=len(test_dataset.tokenizer),
                              tokenized_EOT=test_dataset.tokenized_EOT)
    print("Train only prefix")
else:
    model = ClipCaptionModel(prefix_length, clip_length=prefix_length_clip, prefix_size=prefix_dim,
                              num_layers=num_layers, mapping_type=mapping_type, tokenizer_length=len(test_dataset.tokenizer),
                              tokenized_EOT=test_dataset.tokenized_EOT)
    print("Train both prefix and GPT")


Data size is 243


Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Train both prefix and GPT


In [ ]:
model.load_state_dict(torch.load(model_path, map_location="cpu"))

<All keys matched successfully>

#### Start prediction and evaluation

In [ ]:
index = 0
first_token = test_dataset[0][0]
print(first_token)
first_token = first_token.repeat(bs, 1)
# for data in dataset:
#   if index == 7:
#     break
#   tokens = data[0]
#   mask = data[1]
#   label = data[-1]
#   print("token len",len(tokens))
#   print("mask len",len(mask))
#   print(mask)
#   print(label)
#   print(tokens)

#   # print(label.size())

#   index += 1

tensor([    0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
            0,     0,     0,     0,     0,     0, 50257,    32,   582,   290,
          257,  1310,  2933,   287,  4171,  1204, 35129,   389,   686,  5469,
          257,  7872, 47434,   764, 50258])


In [ ]:
def predict(text, images, model):
  model.eval()

  

In [ ]:
# device = torch.device('cuda:0')
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
test_dataloader = DataLoader(test_dataset, batch_size=bs, shuffle=True, drop_last=True)
print(device)
model = model.to(device)
model.eval()

for idx, (tokens, mask, prefix, label) in enumerate(test_dataloader):

  if idx == 7:
    break

  # tokens = first_token
  tokens, mask, prefix, label = tokens.to(device), mask.to(device), prefix.to(device, dtype=torch.float32), \
                                label.to(device, dtype=torch.float32)


  outputs = model(tokens, prefix, mask, label)

  print("label:",label)
  loss, logits = outputs[:2]


  probability = nnf.softmax(logits).cpu().detach().numpy()

  print(np.round(probability, decimals=3))
  print(loss)


cpu


GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


label: tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])
[[1.    0.   ]
 [1.    0.   ]
 [0.086 0.914]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.004 0.996]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]]
tensor(6.8381, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`


label: tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])
[[1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.002 0.998]
 [1.    0.   ]
 [0.    1.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.    1.   ]
 [1.    0.   ]]
tensor(7.1749, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`


label: tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])
[[1.    0.   ]
 [0.999 0.001]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.    1.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.    1.   ]
 [1.    0.   ]]
tensor(7.2042, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


GPT2ForSequenceClassification will not detect padding tokens in `inputs_embeds`. Results may be unexpected if using padding tokens in conjunction with `inputs_embeds.`


label: tensor([[0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.],
        [0., 1.]])
[[0.    1.   ]
 [0.727 0.273]
 [0.02  0.98 ]
 [0.    1.   ]
 [0.999 0.001]
 [0.    1.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [1.    0.   ]
 [0.897 0.103]
 [1.    0.   ]
 [0.    1.   ]
 [1.    0.   ]]
tensor(4.4694, grad_fn=<BinaryCrossEntropyWithLogitsBackward0>)


KeyboardInterrupt: ignored

In [ ]:
import numpy as np
a = torch.tensor([0.015]).cpu().detach().numpy()
a
np.round(a, decimals=2)

array([0.02], dtype=float32)